In [ ]:
from scipy.optimize import linprog
import json
import math

In [ ]:
def extract_result_from_optim_result(optim, p_num=None):
    arr = optim.x
    result = {"icu": [], "general inpatient": [], "icu waitlist": [], "rejection": [], "allocation error": []}
    # currently, we first processed incoming patient data, then existing patient
    if p_num:
      for i in range(int(len(arr)/4)):
          # first idx means whether to icu
          if arr[i*4] == 1:
              result["icu"].append(p_num[i])
          # second idx means whether to general inpatient unit
          elif arr[i*4+1] == 1:
              result["general inpatient"].append(p_num[i])
          # third idx means whether to icu waitlist
          elif arr[i*4+2] == 1:
              result["icu waitlist"].append(p_num[i])
          elif arr[i*4+3] == 1:
              result["rejection"].append(p_num[i])
          # see if error happens
          else:
              result["allocation error"].append(p_num[i])
    else:
      for i in range(int(len(arr)/4)):
          # first idx means whether to icu
          if arr[i*4] == 1:
              result["icu"].append(i)
          # second idx means whether to general inpatient unit
          elif arr[i*4+1] == 1:
              result["general inpatient"].append(i)
          # third idx means whether to icu waitlist
          elif arr[i*4+2] == 1:
              result["icu waitlist"].append(i)
          elif arr[i*4+3] == 1:
              result["rejection"].append(i)
          # see if error happens
          else:
              result["allocation error"].append(i)
    return result

In [ ]:



def mapping_sofa_to_mortality(sofa_score):
    # statistics of sofa score mapping to mortality rate
    # 0 to 6	< 10%
    # 7 to 9	15 - 35%
    # 10 to 12	40 - 50%
    # 13 to 15	55 - 75%
    # 16 to 24	> 80%
    # we use a linear mapping
    assert 0 <= sofa_score <= 24
    if sofa_score == 0:
        mortality = 0
    if 0 < sofa_score <= 6:
        mortality = 0.1 / sofa_score
    elif 7 <= sofa_score <= 9:
        mortality = 0.1 * sofa_score - 0.55
    elif 10 <= sofa_score <= 12:
        mortality = 0.05 * sofa_score - 0.1
    elif 13 <= sofa_score <= 15:
        mortality = 0.1 * sofa_score - 0.75
    else:
        mortality = 0.025 * sofa_score + 0.4
    return round(mortality, 3)


def calculate_l_p(data, patient_id, Ep_d, noicu_flag = 0):
  # 1 - M(S) + w_d1 * Ep_d1_ICU + w_d2 * Ep_d2_ICU +...
  # status == 0 or 1 or 2
  Ep_d_icu = []
  weights = 0
  for diag in data[patient_id]["icd_code"]:
    weights += 1/int(data[patient_id]["icd_code"][diag])
  sum = 0
  for diag in data[patient_id]["icd_code"]:
    try:
      sum += (1 / weights) * (1 / int(data[patient_id]["icd_code"][diag])) * Ep_d[diag][noicu_flag]
    except KeyError:
      sum += 0
  l_p = min(1 - mapping_sofa_to_mortality(data[patient_id]["SOFA"]) + sum, 1)
  l_p= max(0, l_p)
  return l_p

def normalize_age(data, patient_id):
  return int(data[patient_id]["age"]) / 100

def calculate_v_p(data, patient_id, Ep_d, noicu_flag):
  l_p = calculate_l_p(data, patient_id, Ep_d, noicu_flag)
  return w1 * l_p + w2 * normalize_age(data, patient_id)

def original_status(data, subj):
  incoming = []
  icu_existing = []
  inpatient_existing = []
  for i in range(len(subj)):
    if data[subj[i]]["status"] == 0:
      incoming.append(i)
    if data[subj[i]]["status"] == 1:
      icu_existing.append(i)
    if data[subj[i]]["status"] == 2:
      inpatient_existing.append(i)
  return ("ORIGINAL_ALLOC ---- Incoming: " + str(incoming) + " ICU Existing: " + str(icu_existing)+ " Inpatient Existing: " + str(inpatient_existing))

def mdf_alloc(data, subj, ricu, rgn, p_num = None):
  icu = []
  inpatient = []
  rejected = []
  sofa_dic = {}
  for i in range(len(p_num)):
    if data[subj[p_num[i]]]["SOFA"] not in sofa_dic.keys():
      sofa_dic[data[subj[p_num[i]]]["SOFA"]] = []
    sofa_dic[data[subj[p_num[i]]]["SOFA"]].append(p_num[i])
  sofas = list(sofa_dic.keys())
  sofas.sort()
  sofas = sofas[::-1]
  n = 0
  p = 0
  q = 0
  # print(sofas)
  # print(sofa_dic)
  while n < min(ricu,len(p_num)):
    s_num = len(sofa_dic[sofas[p]])
    icu.append(sofa_dic[sofas[p]][q])
    q += 1
    if q >= s_num:
      p += 1
      q = 0
    n += 1
  if len(p_num)-ricu > 0:
    while n < min(ricu,len(p_num))+ min(rgn,len(p_num)-ricu):
      s_num = len(sofa_dic[sofas[p]])
      inpatient.append(sofa_dic[sofas[p]][q])
      q += 1
      if q >= s_num:
        p += 1
        q = 0
      n += 1
  if len(p_num)-ricu-rgn > 0:
    while n < min(ricu,len(p_num))+ min(rgn,len(p_num)-ricu)+len(p_num)-ricu-rgn:
      s_num = len(sofa_dic[sofas[p]])
      rejected.append(sofa_dic[sofas[p]][q])
      q += 1
      if q >= s_num:
        p += 1
        q = 0
      n += 1
  return [icu, inpatient, rejected]

def real_alloc(data, subj, p_num = None):
  icu = []
  inpatient = []
  if p_num:
    for i in range(len(p_num)):
      if data[subj[p_num[i]]]["result"] == "ICU":
        icu.append(p_num[i])
      if data[subj[p_num[i]]]["result"] == "INPATIENT":
        inpatient.append(p_num[i])
  else:
    for i in range(len(subj)):
      if data[subj[i]]["result"] == "ICU":
        icu.append(i)
      if data[subj[i]]["result"] == "INPATIENT":
        inpatient.append(i)
  return [icu, inpatient, []]

def cal_accuracy(real_alloc, calculated_alloc):
  icu1 = real_alloc[0]
  icu2 = calculated_alloc[0]
  general1 = real_alloc[1]
  general2 = calculated_alloc[1]
  total_num = len(real_alloc[0]) + len(real_alloc[1])
  missed = 0
  for i in icu1:
    if i not in icu2:
      missed += 1
  for i in general1:
    if i not in general2:
      missed += 1
  return (1-missed/total_num)


if __name__ == "__main__":
  fnm = ''
  file = open(fnm+'newindata.json')
  indata = json.load(file)
  file.close()

  file = open(fnm+'newexdata.json')
  exdata = json.load(file)
  file.close()

  file = open(fnm+'epstats.json')
  Ep_d_list = json.load(file)
  file.close()

  alldata = {key: value for (key, value) in (list(indata.items()) + list(exdata.items()))}
  allsids = []
  for sid in alldata:
    allsids.append(sid)
  allpatients = len(allsids)

  fulldata = {}
  sids = []
  patient_num = []
  for i in range(len(allsids)):
    if alldata[allsids[i]]["status"] == 2:
      continue
    sids.append(allsids[i])
    patient_num.append(i)
    fulldata[allsids[i]] = alldata[allsids[i]]

  # icu_exdata = {}
  # for sid in exdata.keys():
  #   if exdata[sid]["status"] == 1:
  #     icu_exdata[sid] = exdata[sid]

  # fulldata = {key: value for (key, value) in (list(indata.items()) + list(exdata.items()))}
  # sids = []
  # for sid in fulldata:
  #   sids.append(sid)
  total = len(sids)
  # print(sids)

  R_icu = 77 # Number of ICU resources (beds)
  R_noicu = 600 # Number of general impatient resources (beds)
  WL_max = 10 # Upper bound of the wait list length
  WL_step = 3 # Iteration step

  t = [] # bed turnaround time of ICU patients reallocation
  Umax = 0.85  # ICU resource max usage (percentage)

  s = [] # SOFA score
  xi = [] # possibilities of incoming patients going to different departments
  xe = [] # possibilities of existing patients going to different departments
  x = [xi, xe]

  w1 = 0.5
  w2 = 0.8

  ew = 0.99
  #0.99 # Vp_WL = ew * Vp_ICU

  R_noicu = max(R_noicu - len(allsids) + len(sids), 0)

  print(original_status(alldata, allsids))
  r_al = real_alloc(fulldata, allsids, patient_num)
  print(("REAL_ALLOC ----ICU: " + str(r_al[0]) + " Inpatient: " + str(r_al[1])))

  mdf_al = mdf_alloc(fulldata, allsids, R_icu*Umax, R_noicu, patient_num)
  print(("MDI_ALLOC ---- ICU: " + str(mdf_al[0]) + " Inpatient: " + str(mdf_al[1]) + " Rejected: " + str(mdf_al[2])))
  print(cal_accuracy(r_al, mdf_al))
  print("\n")


  for wl_len in range(0, WL_max+1, WL_step):
    obj = []
    lhs = []
    lhs.append([1, 0, 0, 0]*total)
    lhs.append([0, 1, 1, 0]*total)
    lhs.append([0, 0, 1, 0]*total)
    lhs.append([0, 0, -1, 0]*total)

    lhs_eq = []
    rhs_eq = []

    rhs = [int(R_icu*Umax), R_noicu, wl_len+WL_step, 1-wl_len]

    bnd = []

    # -ew/((wl_len)+WL_step/2)
    for i in range(total):
      if fulldata[sids[i]]["status"] == 0: # incoming
        obj.append(-1*calculate_v_p(fulldata, sids[i], Ep_d_list, 0))
        obj.append(-1*calculate_v_p(fulldata, sids[i], Ep_d_list, 1))
        obj.append(-ew/((wl_len)+WL_step/2)*calculate_v_p(fulldata, sids[i], Ep_d_list, 0))
        obj.append(0)
      elif fulldata[sids[i]]["status"] == 1: # existing
      # elif fulldata[sids[i]]["status"] == 1 or fulldata[sids[i]]["status"] == 2: 
        obj.append(-1*calculate_v_p(fulldata, sids[i], Ep_d_list, 0)*1.02)
        obj.append(-1*calculate_v_p(fulldata, sids[i], Ep_d_list, 1)*0.98)
        obj.append(-ew/((wl_len)+WL_step/2)*calculate_v_p(fulldata, sids[i], Ep_d_list, 0))
        obj.append(0)

      lhs_eq.append([0]*i*4 + [1, 1, 1, 1] + [0]*(4*(total-i-1)))
      rhs_eq.append(1)

      for i in range(4):
        bnd.append((0,1))

    # optimization1 = linprog(c=obj, A_ub=lhs, b_ub=rhs, A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd)
    # print(optimization1)

    optimization2 = linprog(c=obj, A_ub=lhs, b_ub=rhs, A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd, method='simplex')
    # print(optimization2)
    print("-------------------------------")
    print("Score: ",  -optimization2["fun"])
    result = extract_result_from_optim_result(optimization2, patient_num)
    print("Accuracy: ", cal_accuracy(r_al, [result["icu"], result["general inpatient"]+result["icu waitlist"]]))
    print(result)

ORIGINAL_ALLOC ---- Incoming: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24] ICU Existing: [25, 29, 31, 36, 37, 39, 40, 42, 43, 44, 46, 47, 50, 53, 55, 56, 61, 65, 67, 68, 69, 70, 79, 83] Inpatient Existing: [26, 27, 28, 30, 32, 33, 34, 35, 38, 41, 45, 48, 49, 51, 52, 54, 57, 58, 59, 60, 62, 63, 64, 66, 71, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 84]
REAL_ALLOC ----ICU: [1, 11, 12, 14, 18, 25, 29, 31, 36, 37, 39, 40, 42, 43, 44, 46, 47, 50, 53, 55, 56, 61, 65, 67, 68, 69, 70, 79, 83] Inpatient: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 15, 16, 17, 19, 20, 21, 22, 23, 24]
MDI_ALLOC ---- ICU: [36, 68, 12, 18, 39, 65, 40, 55, 79, 50, 61, 70, 25, 29, 56, 83, 44, 37, 42, 43, 14, 1, 46, 67, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 16, 17, 19, 20, 21, 22, 23, 24, 31, 47, 53, 69] Inpatient: [] Rejected: []
0.5918367346938775


-------------------------------
Score:  41.24353925644299
Accuracy:  0.7551020408163265
{'icu': [1, 3, 4, 5, 12, 13, 14, 15, 16, 